<img style="float: right; height: 60px; padding-left: 20px;" src="_static/NAVIGATE-logo.png"/>

# Analysis of technology diffusion indicators in IAMs

<br>
<img style="float: left; height: 20px; padding-left: 10px;" src="https://img.shields.io/github/license/iiasa/navigate-technology-analysis">
<img style="float: left; height: 20px; padding-left: 10px;" src="https://img.shields.io/badge/python-3.7_|_3.8_|_3.9-blue?logo=python&logoColor=white">
<img style="float: left; height: 20px; padding-left: 10px;" src="https://img.shields.io/badge/code%20style-black-000000.svg">
<br><br>

Copyright (c) 2022 IIASA; this repository is released under the [MIT License](https://github.com/iiasa/navigate-technology-analysis/blob/main/LICENSE).

## Overview

In the NAVIGATE project (Work Package 2), we compared IAMs based on observed characteristics of model results that are closely linked to the specification of technologies in the models, even if details of the implementation in the models differ. In particular, we looked at the growth rate of various energy sector technologies and the implied learning rates, where investment costs for technologies get cheaper as cumulative installed capacity increases. The aim was to identify the ranges of these parameters across models, distinguishing between granular and “lumpy” technologies (i.e., technologies with few but large units, e..g, nuclear power plants). 

This notebook presents a script to easily create plots to compare indicators across models, regions or technologies.


## Requirements

You can install the **pyam** package using the following command -
note the subtle naming difference on [pypi.org](https://pypi.org/project/pyam-iamc/).

```
pip install pyam-iamc
```

[Read the docs](https://pyam-iamc.readthedocs.io/en/stable/install.html) for alternative installation options.

## Acknowledgements

<img style="float: left; height: 40px; padding-right: 10px;" src="_static/EU-logo-300x201.jpg"/>

This project has received funding from the European Union’s Horizon 2020 research
and innovation programme under grant agreement No. 821124 (NAVIGATE).

In [ ]:
import matplotlib.pyplot as plt
import pyam

In [ ]:
df = pyam.IamDataFrame("<path/to/data/file.xlsx")

In [ ]:
# assign meta indicators to use pyam plotting features

In [ ]:
# change the following line to specify the technology
technology = "<technology>"

# the variables to compute the indicators as usually used in recent IAM projects
energy = f"Secondary Energy|Electricity|{technology}"
total = "Secondary Energy|Electricity"
experience = f"Cumulative Capacity|Electricity|{technology}"
performance = f"Capital Cost|Electricity|{technology}"

# change the following 2 lines to select the "columns" of the plot
dimension = "model"
items = df.model

# set up the matplotlib figures
fig_ene, ax_ene = plt.subplots(1, len(items), figsize=(15, 5), sharey=True)
fig_share, ax_share = plt.subplots(1, len(items), figsize=(15, 5), sharey=True)
fig_growth, ax_growth = plt.subplots(1, len(items), figsize=(15, 5), sharey=True)
fig_change, ax_change = plt.subplots(1, len(items), figsize=(15, 5), sharey=True)
fig_learn, ax_learn = plt.subplots(1, len(items), figsize=(15, 5), sharey=True)

# add additional pyam plotting arguments in the dictionary below
_plot_args = dict(fill_between=True, legend=False, title=False)

# iterate of the `items` along the `dimension`
for i, d in enumerate(items):
    
    _plot_df = df.filter(**{dimension: d})
    _ene_df = _plot_df.filter(variable=energy)

    if _ene_df.empty:
        continue

    # plot absolute values of energy generation
    
    _ene_df.plot(ax=ax_ene[i], **_plot_args)
    ax_ene[i].set_title(d)

    # plot share of technology in the energy system
    
    _share_df = (
        _plot_df.divide(energy, total, "Share")
        .multiply("Share", 100, "Share", ignore_units="%")
    )
    _share_df.plot(ax=ax_share[i], **_plot_args)
    
    # plot growth rate of energy generation
    
    _ene_df._data = _ene_df._data[_ene_df._data > 0]
    
    _ene_growth = (
         _ene_df
        .compute.growth_rate({energy: "Growth Rate"})\
        .multiply("Growth Rate", 100, "Growth", ignore_units="%")
    )
    _ene_growth._data = _ene_growth._data[_ene_growth._data < 100]
    _ene_growth.plot(ax=ax_growth[i], **_plot_args)
    
    # plot the annualized rate of change in the share
    
    _share_df._data = _share_df._data[_share_df._data > 0]
    
    _share_growth = (
        _share_df.compute.growth_rate({"Share": "Change of share"})
        .multiply("Change of share", 100, "Change", ignore_units="%")
    )
    _share_growth._data = _share_growth._data[_share_growth._data < 100]
    _share_growth.plot(ax=ax_change[i], **_plot_args)
    
    # plot the learning rate
    
    _df = _plot_df.compute.learning_rate(
        name=f"Learning Rate",
        performance=performance,
        experience=experience,
    )
    
    if not _df.empty:
        _df._data[_df._data < 0] = 0  # remove "-inf" values for plotting
        _df.plot(ax=ax_learn[i], **_plot_args)

# set the y-axis labels
ax_ene[0].set_ylabel("Energy Consumption (EJ/yr)")
ax_share[0].set_ylabel("Share in Electricity Mix (%)")
ax_growth[0].set_ylabel("Change of Energy Consumption (%)")
ax_change[0].set_ylabel("Change of Share in Electricity Mix (%)")
ax_learn[0].set_ylabel("Learning Rate")